In [ ]:
%load_ext watermark
%watermark -a Filippo_Valle -p numpy,pandas,tensorflow,scipy,seaborn,matplotlib -m -g -r -v

In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import tensorflow as tf
from scipy.stats import entropy, hypergeom
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
%load_ext autoreload
%autoreload
from entropy import get_entropy, get_array_entropy
from overlap import get_overlap, get_pval

In [ ]:
directory = "brca/"
os.chdir(directory)

## KL-div

In [ ]:
df_first = pd.read_csv("trisbm/trisbm_level_1_topic-dist.csv", index_col=1).drop("i_doc", 1)
df_first = df_first.divide(df_first.sum(0),1)

df_second = pd.read_csv("trisbm/trisbm_level_1_metadatum-dist.csv", index_col=1).drop("i_doc", 1)

#same subset of stuff
assert((~df_first.index.isin(df_second.index)).sum()==0)
assert((~df_second.index.isin(df_first.index)).sum()==0)

df_second = df_second.reindex(index=df_first.index)
df_second = df_second.divide(df_second.sum(0),1)

In [ ]:
first = tf.convert_to_tensor(df_first.fillna(0).values.T, dtype=tf.float64)
second = tf.convert_to_tensor(df_second.fillna(0).values.T, dtype=tf.float64)
kld_matrix = get_array_entropy(first, second).numpy()

In [ ]:
cm = sns.clustermap(kld_matrix,
            xticklabels=df_second.columns,
            yticklabels=df_first.columns,
            row_cluster=False,
            col_cluster=False)

ax = cm.ax_heatmap
fig = ax.get_figure()
ax.set_ylabel("hSBM", fontsize=35, rotation=90)
ax.set_xlabel("trisbm", fontsize=35, rotation=0)

ax.yaxis.tick_left()
ax.yaxis.set_label_position("left")
ax.tick_params(labelsize=35)

cax = cm.ax_cbar
cax.tick_params(labelsize=30)
cax.set_title("KL-div P(topic|sample)", fontsize=30)
cm.savefig(f"topic_kl.pdf")

## Cluster conservation

In [ ]:
from sklearn.metrics import adjusted_mutual_info_score

In [ ]:
df_first = pd.read_csv("topsbm/topsbm_level_2_clusters.csv")
df_second = pd.read_csv("trisbm/trisbm_level_0_clusters.csv")
assert(np.isin(list(filter(lambda sample: str(sample)!="nan",df_first.values.ravel())), 
        list(filter(lambda sample: str(sample)!="nan",df_second.values.ravel())), invert=True).sum()==0)

In [ ]:
samples = df_first.values.ravel()
samples = list(filter(lambda sample: str(sample)!="nan",samples))

In [ ]:
partition = []
for sample in samples:
    try:
        partition.append((
                df_first.columns[(df_first==sample).any()].values[0].split(" ")[1],
                df_second.columns[(df_second==sample).any()].values[0].split(" ")[1]
                )
            )
    except:
        pass
partition = list(zip(*partition))

In [ ]:
adjusted_mutual_info_score(partition[0],partition[1])

In [ ]:
AMI_levels = [[0.127, 0],
              [0.31, 0],
              [0.25, 0],
              [0.005, 0]]

In [ ]:
cm = sns.clustermap(AMI_levels,
            xticklabels=["l%d"%l for l in range(2)],
            yticklabels=["l%d"%l for l in range(4)],
            row_cluster=False,
            col_cluster=False,
            vmax=1)

ax = cm.ax_heatmap
fig = ax.get_figure()
ax.set_title("clusters", fontsize=35)
ax.set_ylabel("hSBM", fontsize=35, rotation=90)
ax.set_xlabel("trisbm", fontsize=35, rotation=0)

ax.yaxis.tick_left()
ax.yaxis.set_label_position("left")
ax.tick_params(labelsize=35)

cax = cm.ax_cbar
cax.tick_params(labelsize=30)
cax.set_title("AMI", fontsize=30)
plt.tight_layout()
fig.savefig("cluster_overlap_levels.pdf")

In [ ]:
df_cluster_overlap = pd.DataFrame(index=df_first.columns, columns=df_second.columns, data=0.)

In [ ]:
def get_overlap(x,y):
    return np.isin(x,y).sum().astype(float)/float(len(y))

def get_pval(setA, setB):
    x = np.isin(setA,setB).sum() # number of successes
    M = len(samples) # pop size
    k = len(setB) # successes in pop
    N = len(setA) # sample size
    pval = hypergeom.sf(x-1, M, k, N)
    return pval

In [ ]:
for row in df_first.columns:
    for column in df_second.columns:
        df_cluster_overlap.at[row,column]=get_overlap(df_first[row].dropna().values, df_second[column].dropna().values)

In [ ]:
cm = sns.clustermap(df_cluster_overlap,
            xticklabels=df_second.columns,
            yticklabels=df_first.columns,
            row_cluster=False,
            col_cluster=False)

ax = cm.ax_heatmap
fig = ax.get_figure()
ax.set_ylabel("hSBM", fontsize=35, rotation=90)
ax.set_xlabel("trisbm", fontsize=35, rotation=0)

ax.yaxis.tick_left()
ax.yaxis.set_label_position("left")
ax.tick_params(labelsize=35)

cax = cm.ax_cbar
cax.tick_params(labelsize=30)
cax.set_title("%sample\nin common", fontsize=30)
cm.savefig(f"topic_kl.pdf")

## Topic conservation

In [ ]:
df_first = pd.read_csv("topsbm/topsbm_level_1_topics.csv")
df_second = pd.read_csv("trisbm/trisbm_level_1_topics.csv")
assert(np.isin(list(filter(lambda sample: str(sample)!="nan",df_first.values.ravel())), 
        list(filter(lambda sample: str(sample)!="nan",df_second.values.ravel())), invert=True).sum()==0)

In [ ]:
genes = df_first.values.ravel()
genes = list(filter(lambda sample: str(sample)!="nan",genes))

In [ ]:
partition = []
for sample in genes:
    partition.append((
        df_first.columns[(df_first==sample).any()].values[0].split(" ")[1],
        df_second.columns[(df_second==sample).any()].values[0].split(" ")[1]
        )
    )
partition = list(zip(*partition))

In [ ]:
adjusted_mutual_info_score(partition[0], partition[1])

In [ ]:
AMI_levels = [[0.24, 0.099],
             [0.25,0.09],
             [0.12, 0.09],
             [0.08, 0.10]]

In [ ]:
cm = sns.clustermap(AMI_levels,
            xticklabels=["l%d"%l for l in range(2)],
            yticklabels=["l%d"%l for l in range(4)],
            row_cluster=False,
            col_cluster=False,
            vmax=1)

ax = cm.ax_heatmap
fig = ax.get_figure()
ax.set_title("topics", fontsize=35)
ax.set_ylabel("hSBM", fontsize=35, rotation=90)
ax.set_xlabel("trisbm", fontsize=35, rotation=0)

ax.yaxis.tick_left()
ax.yaxis.set_label_position("left")
ax.tick_params(labelsize=35)

cax = cm.ax_cbar
cax.tick_params(labelsize=30)
cax.set_title("AMI", fontsize=30)
plt.tight_layout()
fig.savefig("topic_overlap_levels.pdf")

In [ ]:
df_cluster_overlap = pd.DataFrame(index=df_first.columns, columns=df_second.columns, data=0.)

In [ ]:
for row in df_first.columns:
    for column in df_second.columns:
        df_cluster_overlap.at[row,column]=get_pval(df_first[row].dropna().values, df_second[column].dropna().values)

In [ ]:
cm = sns.clustermap(df_cluster_overlap,
            xticklabels=df_second.columns,
            yticklabels=df_first.columns,
            row_cluster=False,
            col_cluster=False,
            vmax=1)

ax = cm.ax_heatmap
fig = ax.get_figure()
ax.set_ylabel("hSBM", fontsize=35, rotation=90)
ax.set_xlabel("trisbm", fontsize=35, rotation=0)

ax.yaxis.tick_left()
ax.yaxis.set_label_position("left")
ax.tick_params(labelsize=35)

cax = cm.ax_cbar
cax.tick_params(labelsize=30)
#cax.set_title("% gene in common", fontsize=30)
cax.set_title("P_hypergeom", fontsize=30)
cm.savefig(f"topic_overlap.pdf")

In [ ]:
fig, ax = plt.subplots()

df_first.apply(lambda s: (~s.isna()).sum()).hist(ax=ax, histtype="step", lw=15, bins=10)
df_second.apply(lambda s: (~s.isna()).sum()).hist(ax=ax, histtype="step", lw=15, ls="--", bins=10)

ax.set_xlabel("topic size", fontsize=20)

ax.tick_params(labelsize=15)